In [ ]:
# # google colab tesla P100
# ! pip install numpy==1.17.4 scipy==1.3.1 pandas==0.25.3 tensorflow-gpu==2.0.0 torch==1.3.1 torchvision==0.4.2 scikit-learn==0.21.3
# ! pip install git+https://github.com/huggingface/transformers.git@7a9aae1044aa4699310a8004f631fc0a4bdf1b65

In [2]:
# # linux系統指令 可省略 win可能跑不了
# ! nvidia-smi
# ! lscpu
# ! free -h
try:
    import os
    f = os.popen('nvidia-smi')
    f = f.read()
    print(f)
    f = os.popen('lscpu')
    f = f.read()
    print(f)
    f = os.popen('free -h')
    f = f.read()
    print(f)
except:
    pass

Mon Nov 25 16:22:59 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from transformers import BertTokenizer, BertModel, TFBertForSequenceClassification, AdamW

In [4]:
# 讀取tsv # \t tab做區隔
df_news = pd.read_csv("https://github.com/roccqqck/news_bert/raw/master/data/2015_Company_textrank_sentiments.tsv", sep="\t", encoding="utf-8")
df_news['text'] = df_news['text'].astype(str)
df_news['text2'] = df_news['text2'].astype(str)
df_news

,id,label,text,text2,sentiments
0,1,0,財政部充分尊重銀行團確保債權的必要作為，預計元旦後就會請法院實施假扣押，如三重土地順利拍賣，...,银行团发出顶率6財政部充分尊重銀行團確保債權的必要作為，預計元旦後就會請法院實施假扣押，如三...,-0.996732
1,2,0,至三趴以上，「在假扣押前，仍歡迎頂新還錢。」頂新：將持續協商財政部次長吳當傑尊重並同意銀行團...,．银行钱土地三至三趴以上，「在假扣押前，仍歡迎頂新還錢。」頂新：將持續協商財政部次長吳當傑尊...,0.999998
2,3,1,測報告中指出，各款手機在裝上SIM卡前後開機連線時，傳送之資料皆未涉及第一類的敏感資訊，僅為...,资讯资料SIM涉及旗測報告中指出，各款手機在裝上SIM卡前後開機連線時，傳送之資料皆未涉及第...,0.793344
3,4,1,大哥大，透過4G漫遊方式共網爭議，戰火延燒，威寶電信（台灣之星）及中華電信、遠傳電信等業者，...,NCC现行法规电信中华大哥大，透過4G漫遊方式共網爭議，戰火延燒，威寶電信（台灣之星）及中華...,0.979061
4,5,1,測報告中指出，各款手機在裝上SIM卡前後開機連線時，傳送之資料皆未涉及第一類的敏感資訊，僅為...,资讯资料SIM涉及旗測報告中指出，各款手機在裝上SIM卡前後開機連線時，傳送之資料皆未涉及第...,0.793344
...,...,...,...,...,...
16713,16714,3,上市公司玉晶光（3406）今天傳出裁員，對此，公司發言人趙志強表示，並非裁員而 上市公司玉晶...,公司裁员玉并非短期上市公司玉晶光（3406）今天傳出裁員，對此，公司發言人趙志強表示，並非裁...,0.978056
16714,16715,3,繼台企銀（2834）工會因合併問題而罷工，創金融史首例後，大眾銀行（2847）工 繼台企銀（...,工会银企（2847）继台繼台企銀（2834）工會因合併問題而罷工，創金融史首例後，大眾銀行（...,-0.998512
16715,16716,3,6億元賣給英業達，獲利21億元，預計明年首季入帳，活化資產。外界認為這將有機會帶動宏達電明年...,电两首季明年元6億元賣給英業達，獲利21億元，預計明年首季入帳，活化資產。外界認為這將有機會...,0.999983
16716,16717,3,智慧手機大廠宏達電，要迎接新的一年之前，卻傳出讓人遺憾的消息！員工爆料，有兩位中階主管，傳出...,都员工宏达传出长智慧手機大廠宏達電，要迎接新的一年之前，卻傳出讓人遺憾的消息！員工爆料，有兩...,0.999967


In [5]:
df_news['label'].value_counts()

4    5623
0    4877
1    4546
3    1316
2     356
Name: label, dtype: int64

In [ ]:
# 文章字數 > 510了話 去尾
# 字數小於512-2 因為還有CLS SEP
def remove_510(text):
    if len(text) > 510:
        text = text[:510]    # 只取前510個字
    return text

In [7]:
df_news["text"] = df_news["text"].apply(remove_510)
df_news["text2"] = df_news["text2"].apply(remove_510)
df_news.head(2)

,id,label,text,text2,sentiments
0,1,0,財政部充分尊重銀行團確保債權的必要作為，預計元旦後就會請法院實施假扣押，如三重土地順利拍賣，...,银行团发出顶率6財政部充分尊重銀行團確保債權的必要作為，預計元旦後就會請法院實施假扣押，如三...,-0.996732
1,2,0,至三趴以上，「在假扣押前，仍歡迎頂新還錢。」頂新：將持續協商財政部次長吳當傑尊重並同意銀行團...,．银行钱土地三至三趴以上，「在假扣押前，仍歡迎頂新還錢。」頂新：將持續協商財政部次長吳當傑尊...,0.999998


In [8]:
# Load pre-trained model tokenizer, to convert our text into tokens that correspond to BERT’s vocabulary.
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

100%|██████████| 109540/109540 [00:00<00:00, 650851.75B/s]


![avatar](https://github.com/roccqqck/news_bert/raw/master/bert_input_encoding.jpg)

https://github.com/roccqqck/news_bert/raw/master/bert_input_encoding.jpg

bert input features 有3個

input_ids: 代表識別每個 token 的索引值，用 tokenizer 轉換即可

token_type_ids: 用來識別句子界限。第一句為 0，第二句則為 1。另外注意句子間的 [SEP] 為 0     (optional) 輸入有1句非必要 輸入有2句則必要

attention_mask: 用來界定自注意力機制範圍。1 讓 BERT 關注該位置，0 則代表是 padding 不需關注
(optional)

https://huggingface.co/transformers/model_doc/bert.html#tfbertforsequenceclassification

In [9]:
# Tokenize input
text = "很好看的動作片，不會浪費錢跟時間。很久沒有這樣的探險片。可說是女版的印第安那瓊。"
tokens = tokenizer.tokenize(text)      # 每個字切詞成一個list
print(type(tokens))                 # list
np.array(tokens)                    # 轉成numpy

<class 'list'>


array(['很', '好', '看', '的', '動', '作', '片', '，', '不', '會', '浪', '費', '錢',
       '跟', '時', '間', '。', '很', '久', '沒', '有', '這', '樣', '的', '探', '險',
       '片', '。', '可', '說', '是', '女', '版', '的', '印', '第', '安', '那', '瓊',
       '。'], dtype='<U1')

In [10]:
input_ids = tokenizer.convert_tokens_to_ids(tokens)   # 每個字轉成id
print(type(input_ids))                         # list
print(len(input_ids))
np.array(input_ids)

<class 'list'>
40


array([2523, 1962, 4692, 4638, 1240,  868, 4275, 8024,  679, 3298, 3857,
       6527, 7092, 6656, 3229, 7279,  511, 2523,  719, 3760, 3300, 6857,
       3564, 4638, 2968, 7402, 4275,  511, 1377, 6303, 3221, 1957, 4276,
       4638, 1313, 5018, 2128, 6929, 4475,  511])

In [11]:
token_type_ids = tokenizer.create_token_type_ids_from_sequences(input_ids) # token_type_ids 必須input還沒加CLS SEP
print(type(token_type_ids))                                # list
print(len(token_type_ids)) 
np.array(token_type_ids)

<class 'list'>
42


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)    # 句子前後加上 CLS SEP 的 id
print(type(input_ids))
print(len(input_ids))
np.array(input_ids)

<class 'list'>
42


array([ 101, 2523, 1962, 4692, 4638, 1240,  868, 4275, 8024,  679, 3298,
       3857, 6527, 7092, 6656, 3229, 7279,  511, 2523,  719, 3760, 3300,
       6857, 3564, 4638, 2968, 7402, 4275,  511, 1377, 6303, 3221, 1957,
       4276, 4638, 1313, 5018, 2128, 6929, 4475,  511,  102])

In [13]:
n = 512 - len(input_ids)
input_ids2 = np.pad(input_ids, (0, n), mode ='constant', constant_values=(0))  
# array右邊append n 個 0  補長度到512
print(len(input_ids2))
input_ids2

512


array([ 101, 2523, 1962, 4692, 4638, 1240,  868, 4275, 8024,  679, 3298,
       3857, 6527, 7092, 6656, 3229, 7279,  511, 2523,  719, 3760, 3300,
       6857, 3564, 4638, 2968, 7402, 4275,  511, 1377, 6303, 3221, 1957,
       4276, 4638, 1313, 5018, 2128, 6929, 4475,  511,  102,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [14]:
# input如果是兩個句子

text = "很好看的動作片"
tokens = tokenizer.tokenize(text)      # 每個字切詞成一個list
print(type(tokens))                 # list
np.array(tokens)                    # 轉成numpy

<class 'list'>


array(['很', '好', '看', '的', '動', '作', '片'], dtype='<U1')

In [15]:
text2 = "不會浪費錢跟時間"
tokens2 = tokenizer.tokenize(text2)      # 每個字切詞成一個list
print(type(tokens2))                 # list
np.array(tokens2)                    # 轉成numpy

<class 'list'>


array(['不', '會', '浪', '費', '錢', '跟', '時', '間'], dtype='<U1')

In [16]:
input_ids = tokenizer.convert_tokens_to_ids(tokens)   # 每個字轉成id
print(type(input_ids))                         # list
print(len(input_ids))
np.array(input_ids)

<class 'list'>
7


array([2523, 1962, 4692, 4638, 1240,  868, 4275])

In [17]:
input_ids2 = tokenizer.convert_tokens_to_ids(tokens2)   # 每個字轉成id
print(type(input_ids2))                         # list
print(len(input_ids2))
np.array(input_ids2)

<class 'list'>
8


array([ 679, 3298, 3857, 6527, 7092, 6656, 3229, 7279])

In [18]:
token_type_ids = tokenizer.create_token_type_ids_from_sequences(input_ids, input_ids2) # token_type_ids 必須input還沒加CLS SEP
print(type(token_type_ids))                                # list
print(len(token_type_ids)) 
np.array(token_type_ids)

<class 'list'>
18


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [19]:
input_ids3 = tokenizer.build_inputs_with_special_tokens(input_ids, input_ids2)    # 句子前後加上 CLS SEP 的 id
print(type(input_ids3))
print(len(input_ids3))
np.array(input_ids3)

<class 'list'>
18


array([ 101, 2523, 1962, 4692, 4638, 1240,  868, 4275,  102,  679, 3298,
       3857, 6527, 7092, 6656, 3229, 7279,  102])

numpy.pad   補0到某長度

https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html 

也可以使用

```from keras.preprocessing.sequence import pad_sequences```


In [ ]:
def input_ids_all(text):
#    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    tokens = tokenizer.tokenize(text)        # 每個字切詞成一個list
    input_ids = tokenizer.convert_tokens_to_ids(tokens)  # 每個字轉成id
    input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)    # 句子前後加上 CLS SEP 的 id
    input_ids = np.array(input_ids)          # list 轉 numpy
    if len(input_ids) < 512:
        n = 512 - len(input_ids)
        input_ids = np.pad(input_ids, (0, n), mode ='constant', constant_values=(0))  
        # array右邊append n 個 0  補長度到512
    return input_ids

In [21]:
text = "很好看的動作片，不會浪費錢跟時間。很久沒有這樣的探險片。可說是女版的印第安那瓊。"
input_ids_all(text)

array([ 101, 2523, 1962, 4692, 4638, 1240,  868, 4275, 8024,  679, 3298,
       3857, 6527, 7092, 6656, 3229, 7279,  511, 2523,  719, 3760, 3300,
       6857, 3564, 4638, 2968, 7402, 4275,  511, 1377, 6303, 3221, 1957,
       4276, 4638, 1313, 5018, 2128, 6929, 4475,  511,  102,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [ ]:
def attention_mask_all(text):
    tokens = tokenizer.tokenize(text)       # 每個字切詞成一個list
    input_ids = tokenizer.convert_tokens_to_ids(tokens)  # 每個字轉成id
    input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)    # 句子前後加上 CLS SEP 的 id
    input_ids = np.array(input_ids)          # list 轉 numpy
    attention_mask = np.array([1,1])
    attention_mask = np.pad(attention_mask, (0, len(input_ids)-2 ), mode ='constant', constant_values=(1)) 
    # array右邊append 1 到跟segment一樣長
    if len(attention_mask) < 512:
        n = 512 - len(attention_mask)
        attention_mask = np.pad(attention_mask, (0, n), mode ='constant', constant_values=(0))  # array右邊append n 個 0  補長度到512
    return attention_mask

In [23]:
attention_mask_all(text)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# 事實上單一句子　出來都是0　不做也沒差

def token_type_ids_all(text):
    tokens = tokenizer.tokenize(text)       # 每個字切詞成一個list
    input_ids = tokenizer.convert_tokens_to_ids(tokens)  # 每個字轉成id
    input_ids = np.array(input_ids)          # list 轉 numpy
    token_type_ids = tokenizer.create_token_type_ids_from_sequences(input_ids)   # token_type_ids 必須input還沒加CLS SEP
    token_type_ids = np.array(token_type_ids)              # list 轉numpy
    if len(token_type_ids) < 512:
        n = 512 - len(token_type_ids)
        token_type_ids = np.pad(token_type_ids, (0, n), mode ='constant', constant_values=(0))  
        # array右邊append n 個 0  補長度到512    
    return token_type_ids

In [25]:
token_type_ids_all(text)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

最後我決定用pandas的apply 比較好視覺化理解

In [26]:
# df['text2'] = df['text']].apply(lambda x: " ".join(jieba.cut(x)))
# df_news['tokens'] = df_news['text'].apply(lambda x:  tokenizer.tokenize(x) )
df_news['input_ids'] = df_news['text2'].apply(input_ids_all)
df_news['attention_mask'] = df_news['text2'].apply(attention_mask_all)
df_news['token_type_ids'] = df_news['text2'].apply(token_type_ids_all)
df_news.head(2)

,id,label,text,text2,sentiments,input_ids,attention_mask,token_type_ids
0,1,0,財政部充分尊重銀行團確保債權的必要作為，預計元旦後就會請法院實施假扣押，如三重土地順利拍賣，...,银行团发出顶率6財政部充分尊重銀行團確保債權的必要作為，預計元旦後就會請法院實施假扣押，如三...,-0.996732,"[101, 7213, 6121, 1730, 1355, 1139, 7553, 4372...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,0,至三趴以上，「在假扣押前，仍歡迎頂新還錢。」頂新：將持續協商財政部次長吳當傑尊重並同意銀行團...,．银行钱土地三至三趴以上，「在假扣押前，仍歡迎頂新還錢。」頂新：將持續協商財政部次長吳當傑尊...,0.999998,"[101, 8026, 7213, 6121, 7178, 1759, 1765, 676,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
# df_news['input_ids'].to_numpy()   # 提出來 竟然不是2d numpy 不能這樣做

In [28]:
print(len(df_news['input_ids'][0]))
df_news['input_ids'][0] 

512


array([ 101, 7213, 6121, 1730, 1355, 1139, 7553, 4372,  127, 6512, 3124,
       6956, 1041, 1146, 2203, 7028, 7065, 6121, 1757, 4825,  924, 1002,
       3609, 4638, 2553, 6206,  868, 4158, 8024, 7521, 6243, 1039, 3190,
       2527, 2218, 3298, 6313, 3791, 7368, 2179, 3177,  969, 2807, 2852,
       8024, 1963,  676, 7028, 1759, 1765, 7518, 1164, 2864, 6546, 8024,
       7515, 4372, 6917, 3621, 2200,  679, 2512, 7513, 1456, 1059,  127,
       1283, 1399, 1519, 2339, 4638, 4495, 6243,  511, 1042, 6493, 7065,
       6121, 6134, 4850, 8024, 7515, 4372, 7274, 4634, 5474, 6526, 2234,
       3309, 6889, 5147, 3428, 8024, 5195, 5474, 6526, 7065, 6121, 1757,
       3748, 6359, 4634, 1139,  998, 1440, 2100, 6349,  928, 1141, 8024,
       6313,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [29]:
print(len(df_news['token_type_ids'][0]))
df_news['token_type_ids'][0]

512


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [30]:
print(len(df_news['attention_mask'][0]))
df_news['attention_mask'][0]

512


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# # df_news['input_ids'].to_numpy() 出來不是一個2d numpy 
# # 只好用for loop一個一個拿出來合併
# # 用np vstack超級慢 不知道為何  改用最外層是list append

# input_ids = np.zeros((1, 512)).astype(int) #宣吿一個都是0的1*512 numpy # np.zeros預設是float 改成int 不然bert餵不進去
# for index, row in df_news.iterrows():  
#     element = df_news.loc[index,'input_ids']
#     input_ids = np.vstack((input_ids, np.array([element])))   # 2維 合併

# input_ids = np.delete(input_ids, 0, 0)              # 刪掉一開始都是0的那一個宣告  
# input_ids

https://www.quora.com/Is-it-better-to-use-np-append-or-list-append

In [32]:
# df_news['input_ids'].to_numpy() 出來不是一個2d numpy 
# 只好用for loop一個一個拿出來合併
# 用np vstack超級慢 不知道為何  改用最外層是list append
input_ids = []      # list
for index, row in df_news.iterrows():  
    np_1d = df_news.loc[index,'input_ids']    # 1d np arrary
    input_ids.append(np_1d)                       # 1d np的 list # list[np_1, np_2, np_3, ....]


input_ids = np.array(input_ids)                       # 轉成2d np
input_ids

array([[ 101, 7213, 6121, ...,    0,    0,    0],
       [ 101, 8026, 7213, ...,    0,    0,    0],
       [ 101, 6598, 6380, ...,    0,    0,    0],
       ...,
       [ 101, 4510,  697, ...,    0,    0,    0],
       [ 101, 6963, 1447, ...,    0,    0,    0],
       [ 101, 2131, 6809, ...,    0,    0,    0]])

In [33]:
input_ids.shape

(16718, 512)

In [ ]:
# torch.tensor(input_ids)    # numpy 轉 torch tensor

In [35]:
tf.convert_to_tensor(input_ids)   # numpy 轉 tf tensor

<tf.Tensor: id=2, shape=(16718, 512), dtype=int64, numpy=
array([[ 101, 7213, 6121, ...,    0,    0,    0],
       [ 101, 8026, 7213, ...,    0,    0,    0],
       [ 101, 6598, 6380, ...,    0,    0,    0],
       ...,
       [ 101, 4510,  697, ...,    0,    0,    0],
       [ 101, 6963, 1447, ...,    0,    0,    0],
       [ 101, 2131, 6809, ...,    0,    0,    0]])>

In [36]:
attention_mask = []      # list
for index, row in df_news.iterrows():  
    np_1d = df_news.loc[index,'attention_mask']    # 1d np arrary
    attention_mask.append(np_1d)                       # 1d np的 list # list[np_1, np_2, np_3, ....]


attention_mask = np.array(attention_mask)                       # 轉成2d np
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [37]:
attention_mask.shape

(16718, 512)

In [38]:
token_type_ids = []      # list
for index, row in df_news.iterrows():  
    np_1d = df_news.loc[index,'token_type_ids']    # 1d np arrary
    token_type_ids.append(np_1d)                       # 1d np的 list # list[np_1, np_2, np_3, ....]


token_type_ids = np.array(token_type_ids)                       # 轉成2d np
token_type_ids

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [39]:
token_type_ids.shape

(16718, 512)

In [40]:
label = df_news['label'].to_numpy()
label

array([0, 0, 1, ..., 3, 3, 3])

In [ ]:
# 把2個input_ids, attention_mask , token_type_ids 還有label 切成training data, validation data

from sklearn.model_selection import train_test_split
# Use train_test_split to split our data into train and validation sets for training

# # 設定 stratify = label 把每個類別平均
train_input_ids, validation_input_ids, train_label, validation_label = train_test_split(input_ids, label, 
                                                            random_state=2018, test_size=0.5, stratify=label )

train_attention_mask, validation_attention_mask, _, _ = train_test_split(attention_mask, label,
                                             random_state=2018, test_size=0.5, stratify=label )

train_token_type_ids, validation_token_type_ids, _, _ = train_test_split(token_type_ids, label,
                                             random_state=2018, test_size=0.5, stratify=label )

In [ ]:
# # input 可用 numpy 或 tf tensor 下面是numpy轉tf tensor

# train_input_ids = tf.convert_to_tensor(train_input_ids)
# validation_input_ids = tf.convert_to_tensor(validation_input_ids)
# train_label = tf.convert_to_tensor(train_label)
# validation_label = tf.convert_to_tensor(validation_label)
# train_attention_mask = tf.convert_to_tensor(train_attention_mask)
# validation_attention_mask = tf.convert_to_tensor(validation_attention_mask)
# train_token_type_ids = tf.convert_to_tensor(train_token_type_ids)
# validation_token_type_ids = tf.convert_to_tensor(validation_token_type_ids)

In [43]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule 
# num_labels=5 分5類
model = TFBertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=5)
model.summary()


optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

100%|██████████| 478309336/478309336 [00:15<00:00, 30945103.79B/s]


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  102267648 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3845      
Total params: 102,271,493
Trainable params: 102,271,493
Non-trainable params: 0
_________________________________________________________________


In [44]:
%%time
# Train and evaluate using tf.keras.Model.fit()  # batch size 16就會error 是GPU記憶體爆掉
model_fit = model.fit(train_input_ids, train_label, 
                      batch_size=8, epochs=4, 
                    validation_data=(validation_input_ids, validation_label)
#                    steps_per_epoch=115,
#                    validation_steps=7)
                   )

Train on 8359 samples, validate on 8359 samples
Epoch 1/4
8359/8359 [==============================] - 804s 96ms/sample - loss: 0.3144 - accuracy: 0.8944 - val_loss: 0.1503 - val_accuracy: 0.9498
Epoch 2/4
8359/8359 [==============================] - 784s 94ms/sample - loss: 0.1303 - accuracy: 0.9605 - val_loss: 0.1312 - val_accuracy: 0.9585
Epoch 3/4
8359/8359 [==============================] - 782s 94ms/sample - loss: 0.1010 - accuracy: 0.9684 - val_loss: 0.1096 - val_accuracy: 0.9690
Epoch 4/4
8359/8359 [==============================] - 781s 93ms/sample - loss: 0.0785 - accuracy: 0.9742 - val_loss: 0.1035 - val_accuracy: 0.9654
CPU times: user 17min 17s, sys: 6min 46s, total: 24min 4s
Wall time: 52min 35s


https://huggingface.co/transformers/model_doc/bert.html#tfbertforsequenceclassification

attention_mask 或 token_type_ids 不一定要放輸入 可選擇

要放了話要加```[ ]```

```model.fit([train_input_ids, train_attention_mask, train_token_type_ids], train_label)```

就是```model.fit(X_train, Y_train)```

```[train_input_ids, train_attention_mask, train_token_type_ids]``` 就是 ```X_train```

```train_label``` 就是 ```Y_train```

In [ ]:
# model_fit = model.fit([train_input_ids, train_attention_mask, train_token_type_ids], train_label, 
#                        batch_size=4, epochs=1, 
#                     validation_data=([validation_input_ids, validation_attention_mask, validation_token_type_ids], validation_label)
#                    )

In [ ]:
# # 如果train到一半 想要重新train 在jupyter裡面interrupt kernel
# # 這時候 model還是存在在記憶體裡面 只是train到一半 要重新train要釋放model的記憶體
# del model

In [ ]:
# model.evaluate(validation_input_ids, validation_label, verbose=1)

https://blog.csdn.net/zds13257177985/article/details/80638384

```predictions = model.predict（test）```預測的是數值，而且輸出的是n*5的編碼值array

要經過```predictions = np.argmax(predictions, axis=1)```才是類別

In [48]:
%%time
predictions = model.predict(validation_input_ids)   # 輸出的是n*5的編碼值array
predictions

CPU times: user 26.8 s, sys: 8.54 s, total: 35.3 s
Wall time: 2min 54s


In [49]:
predictions = np.argmax(predictions, axis=1)         # axis = 1是取行的最大值的索引，0是列的最大值的索引
predictions

array([4, 1, 3, ..., 0, 0, 0])

In [50]:
from sklearn.metrics import accuracy_score
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import f1_score
# from sklearn.metrics import cohen_kappa_score
# from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(accuracy_score(validation_label, predictions))
# print(precision_score(validation_label, predictions))
# print(recall_score(validation_label, predictions))
# print(f1_score(validation_label, predictions))
print(confusion_matrix(validation_label, predictions))
print(classification_report(validation_label, predictions))

0.9654264864218208
[[2432    3    0    0    4]
 [   7 2240    4    7   15]
 [   0    4  172    0    2]
 [   8   18    0  626    6]
 [  10  193    8    0 2600]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2439
           1       0.91      0.99      0.95      2273
           2       0.93      0.97      0.95       178
           3       0.99      0.95      0.97       658
           4       0.99      0.92      0.96      2811

    accuracy                           0.97      8359
   macro avg       0.96      0.97      0.96      8359
weighted avg       0.97      0.97      0.97      8359

